In [ ]:
import matplotlib
%matplotlib widget
import matplotlib.pyplot as plt
import h5py
import numpy as np
from pathlib import Path
import skimage as sk
from skimage import io as skio
import json
import pandas as pd

import flammkuchen as fl
from split_dataset import SplitDataset
from bouterin.plots.stimulus_log_plot import get_paint_function
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree, to_tree, set_link_color_palette
from sklearn.cluster import AgglomerativeClustering
from fimpylab.core.twop_experiment import TwoPExperiment


In [ ]:
master = Path(r"\\Funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\habenula\fixed")
fish_list = list(master.glob("*f[0-9]*"))
num_fish = len(fish_list)
print(len(fish_list))

In [ ]:
'''
# fix fish0 in habenula folder:
from scipy.interpolate import interp1d 

dir_traces = fish_list[0] / "traces.h5"
traces = fl.load(dir_traces)['traces']
    
t_original = np.arange(0, np.shape(traces)[1]) / 2
t_new = np.arange(0, (np.shape(traces)[1]) * 3/2) / 3

fixed_f0 = interp1d(t_original, traces, bounds_error=False)(t_new)
np.shape(fixed_f0)

dir_traces1 =  fish_list[0] / "traces_original.h5"
fl.save(dir_traces1, {"traces_original": traces})
dir_traces2 =  fish_list[0] / "traces.h5"
fl.save(dir_traces2, {"traces": fixed_f0})
'''

## Cluster all traces:

In [ ]:
# loading all traces
len_segment = 2295
traces_all = 0
for i in range(num_fish):
    f =  fish_list[i]
    print(f)
    dir_traces = f / "traces.h5"
    traces_tmp = fl.load(dir_traces)['traces']
    print(np.shape(traces_tmp))
    traces_tmp = traces_tmp[:, 0:len_segment]
    if traces_all is 0:
        traces_all = traces_tmp
    else:
        traces_all = np.concatenate((traces_all, traces_tmp), axis=0)
    print(np.shape(traces_all))
    
# normalizing traces:
traces_all = traces_all.T
traces_all = ((traces_all - traces_all.mean(0)) / traces_all.std(0))
norm_traces = traces_all.T

In [ ]:
avg_traces = np.zeros((np.shape(norm_traces)[0], np.shape(norm_traces)[1]//3))
for i in range(3):
    t1 = i * np.shape(norm_traces)[1]//3
    t2 = t1 + np.shape(norm_traces)[1]//3
    avg_traces += norm_traces[:, t1:t2]
avg_traces /= 3
print(np.shape(avg_traces))

In [ ]:
avg_traces = np.zeros((3, np.shape(norm_traces)[0], np.shape(norm_traces)[1]//3))
for i in range(3):
    t1 = i * np.shape(norm_traces)[1]//3
    t2 = t1 + np.shape(norm_traces)[1]//3
    avg_traces[i] = norm_traces[:, t1:t2]
avg_traces = np.nanmean(avg_traces, axis=0)
print(np.shape(avg_traces))

In [ ]:
np.shape(np.where(np.isnan(avg_traces)))
np.where(np.isnan(avg_traces))

In [ ]:
########## remove bad traces from lasr fish

#tmp_traces = avg_traces[0:3182]
tmp_traces = np.copy(avg_traces)
#tmp_traces[3180:3208] = np.ones((28, 765))
tmp_traces[3182] = np.ones((1, 765))
tmp_traces[3207] = np.ones((1, 765))

traces_norm_fixed = np.zeros((np.shape(avg_traces)[0]-2, np.shape(avg_traces)[1]))
traces_norm_fixed[0:3182] = avg_traces[0:3182]
traces_norm_fixed[3182:3206] = avg_traces[3183:3207]
traces_norm_fixed[3206:] = avg_traces[3208:]

In [ ]:
fig1 = plt.figure(figsize=(10,7))
linked = linkage(traces_norm_fixed, method='ward')
dend = dendrogram(linked)
plt.show()
#fig1.savefig(str(master / "dendrogrm_210104.jpg"))

In [ ]:
cluster = AgglomerativeClustering(n_clusters=12, affinity='euclidean', linkage='ward')
her_clustering = cluster.fit_predict(traces_norm_fixed)

### From Ot & Luigi

In [ ]:
def cluster_id_search(tree):
    nodes_list = []
    if tree.is_leaf():
        nodes_list.append(tree.get_id())
    else:
        nodes_list += cluster_id_search(tree.get_left())
        nodes_list += cluster_id_search(tree.get_right())
        
    return nodes_list

def find_trunc_dendro_clusters(linkage_mat, dendro):
    tree, branches = to_tree(linkage_mat, rd =True)
    ids = np.empty(linkage_mat.shape[0]+1, dtype=int)
    
    for i, clust in enumerate(dendro["leaves"]):
        ids[cluster_id_search(branches[clust])] = i
        
    return ids

In [ ]:
# make truncated tree to get clusters ids. 
# Ugly but necessary to get the same sequence of leaves as the cut.
plt.figure(figsize=(0.1, 0.1))  
n_clust = 8
dendro = dendrogram(linked, n_clust, truncate_mode ="lastp")
plt.close()
cluster_ids = dendro["leaves"]
labels = find_trunc_dendro_clusters(linked, dendro)

In [ ]:
meanresps = norm_traces
base_sub_mean = (meanresps.T - np.nanmean(meanresps[:,:8], 1)).T
X = base_sub_mean
#smooth_mean_resps = pd.DataFrame(meanresps.T).rolling(4, center=True).mean().as_matrix().T
'''
num_traces = np.shape(norm_traces)[0]
traces_fixed = np.copy(norm_traces)
traces_fixed[np.where(traces_fixed == 0)[0]] = None
for i in range(num_traces):
    tmp_cluster = traces_fixed[i]
    tmp_cluster[np.where(tmp_cluster <= (np.min(tmp_cluster)+0.1))[0]] = None
    #tmp_cluster = tmp_cluster + (i*5)
    
meanresps = traces_fixed
'''

In [ ]:
from skimage import color
import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, cut_tree, set_link_color_palette
#import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm

In [ ]:
def shade_plot(stim, ax=None, gamma=1/6, shade_range=(0.6, 0.9)):
    if type(stim) == list:  # these would be transitions
        _shade_plot(stim, ax=ax, gamma=gamma, shade_range=shade_range)

    elif type(stim) == Data:  # fish data
        transitions = find_transitions(Data.resampled_stim, Data.time_im_rep)
        _shade_plot(transitions, ax=ax, gamma=gamma, shade_range=shade_range)

    elif type(stim) == np.ndarray:  # stimulus array
        transitions = find_transitions(stim[:,1], stim[:,0])
        _shade_plot(transitions, ax=ax, gamma=gamma, shade_range=shade_range)

    elif type(stim) == tuple:  # time, lum tuple
        transitions = find_transitions(stim[1], stim[0])
        _shade_plot(transitions, ax=ax, gamma=gamma, shade_range=shade_range)


def _shade_plot(lum_transitions, ax=None, gamma=1/6, shade_range=(0.6, 0.9)):

    if ax is None:
        ax = plt.gca()
    shade = lum_transitions[0][1]
    for i in range(len(lum_transitions)-1):
        shade = shade + lum_transitions[i][1]
        new_shade = shade_range[0] + np.power(np.abs(shade), gamma) * (shade_range[1] - shade_range[0])
        ax.axvspan(lum_transitions[i][0], lum_transitions[i+1][0], color=(new_shade, )*3)
        

def _find_thr(linked, n_clust):
    interval = [0, 2000]
    new_height = np.mean(interval)
    clust = 0
    n_clust = n_clust
    while clust != n_clust:
        new_height = np.mean(interval)
        clust = cut_tree(linked, height=new_height).max()
        if clust > n_clust:
            interval[0] = new_height
        elif clust < n_clust:
            interval[1] = new_height


    return new_height


def find_plot_thr(linked, n_clust):
    min_thr = _find_thr(linked, n_clust - 1)
    return min_thr  


In [ ]:
def plot_clusters_dendro(traces, stim, linkage_mat, labels, dendrolims=(900, 30),
                         thr=None, f_lim=1.5, gamma=1):
    fig_clust, ax = plt.subplots(3, 1, figsize=(15, 15))
    hexac = cluster_cols()

    n_clust = labels.max() + 1

    ##################
    ### Dendrogram ###
    # Compute and plot first dendrogram.
    if thr is None:
        thr = find_plot_thr(linkage_mat, n_clust)

    set_link_color_palette(hexac)
    ax_traces = plt.subplot2grid((2, 2), (0, 0))
    ax_clusters = plt.subplot2grid((2, 2), (0, 1))
    ax_dendro = plt.subplot2grid((2, 2), (1, 0), colspan=2)

    #ax_dendro = ax[2]
    #ax_traces = ax[1]
    #ax_clusters = ax[0]
    
    panel_dendro = dendrogram(linkage_mat,
                              color_threshold=thr,
                              #orientation='left',
                              distance_sort='descending',
                              show_leaf_counts=False,
                              no_labels=True,
                              above_threshold_color='#%02x%02x%02x' % (
                              120, 120, 120))
    
    ax_dendro.axhline(thr, linewidth=0.7, color="k")
    ax_dendro.axis("off")

    # Plot traces matrix.
    im = ax_traces.imshow(traces[panel_dendro["leaves"], :],
                         aspect='auto', origin='lower', cmap=cm.RdBu_r,
                         vmin=-f_lim, vmax=f_lim)
    ax_traces.axes.spines['left'].set_visible(False)
    ax_traces.set_yticks([])

    # Time bar:
    dt = stim[1, 0]
    barlength = 10
    bounds = np.array([traces.shape[1] - barlength / dt,
                       traces.shape[1]])

    ##################
    # Cluster sizes ##
    # Calculate size of each defined cluster to put colored labels on the side.
    # Find indervals spanned by each cluster in the sorted traces matrix.
    # Add percentages spanned by each cluster.
    sizes = np.cumsum(np.array([np.sum(labels == i) for i in range(np.max(labels) + 1)]))
    intervals = np.insert(sizes, 0, 0)

    ##################
    # Cluster means ##

    for i in range(n_clust):
        ax_clusters.plot(np.nanmean(traces[labels == i, :], 0) +
                      i * 2, label=i, color=hexac[i])
    ax_clusters.axes.spines['left'].set_visible(False)
    ax_clusters.set_yticks([])

    barlength = 10
    ax_traces.axis("off")
    ax_clusters.axis("off")

    return fig_clust, ax_clusters

def cluster_cols():
    # color_list = ["lightblue", "lightcoral", "orange", "springgreen", "deepskyblue", "mediumpurple","gold", "cyan", "crimson", "deeppink", "lawngreen", "darkviolet", "orchid", "limegreen", "seagreen", "chocolate", "blue", "navy"]
    color_list = ["#cc566a", "#cd6c39", "#a39440", "#64ac48", "#4aac8d", "#688bcd", "#8562cc", "#c361aa"]
    # color_list = ["#ff5c67", "#af0006", "#ffa468", "#8c5f00", "#e4a400", "#d5c86f", "#939400", "#a7d380", "#138b00", "#42e087", "#00a86d", "#81c7a8", "#019a82", "#1eaaff", "#0268bb", "#5951d7", "#6b4570", "#ad20aa", "#ffa1e2", "#ff4a94"]
    return color_list

In [ ]:
stim = np.asarray([[1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7]])
fig_clust, ax_clust = plot_clusters_dendro(traces_norm_fixed, stim, linked, labels)#, dendrolims=(940, 0))

f = master / fish_list[0]
exp = TwoPExperiment(path=f)
fs = 3
stimulus_log = exp.load_session_log(log_name='stimulus_log', session_idx=0)
stim_value, t_values = get_paint_function(stimulus_log, 'E0040_motions_cardinal')
stim_value = stim_value / 255
num_stim = np.shape(stim_value)[0] // 3

t_values *= fs
for i in range(num_stim):
    ax_clust.axvspan(
        t_values[i, 0],
        t_values[i, 1],
        facecolor=[
            stim_value[i, 0],
            stim_value[i, 1],
            stim_value[i, 2],
        ],
        alpha=0.3,
    )
    
plt.show()
file_name = 'hierarchical_clustering_210715_k' + str(n_clust) + '.jpg'
fig_clust.savefig(str(master/file_name), dpi=300)

In [ ]:
print(labels)
print(np.unique(labels))
print(np.max(roi_map))

### Create a figure of all ROIs colored by cluster:

In [ ]:
### getting the indices for each fish:
print(num_fish)
num_rois = np.zeros(num_fish)
for i in range(num_fish):
    f = master / fish_list[i]
    dir_traces = f / "traces.h5"
    traces = fl.load(dir_traces)['traces']
    num_traces = np.shape(traces)[0]
    num_rois[i] = num_traces // 1
    print(num_rois)

In [ ]:
ind2 = np.cumsum(num_rois).astype(int)
ind2

In [ ]:
num_rois= num_rois.astype(int)

In [ ]:
ind1 = np.zeros(num_fish+1, dtype=int)
for i in range(num_fish):
    ind1[i+1] = ind2[i]
ind1

In [ ]:
current_fish = 7
fish_dir = master / fish_list[current_fish]
# Rois
rois = fl.load(fish_dir / "merged_rois.h5")["stack"][:,:,:]
labels_fish = labels[ind1[current_fish]:ind2[current_fish]]

In [ ]:
roi_map = np.copy(rois)

roi_map_clustered = np.zeros_like(roi_map)
for i in range(0, num_rois[current_fish]):
    roi_map_clustered[np.where(roi_map == (i + 1))] = labels_fish[i] + 1


In [ ]:
fig1, ax1 = plt.subplots(3, 4, figsize=(12, 12))

color_list = ["lightblue", "lightcoral", "orange", "springgreen", "deepskyblue", "mediumpurple","gold", "cyan", "crimson", "deeppink", "lawngreen", "darkviolet"][0:n_clust]
color_list = ["#ff5c67", "#af0006", "#ffa468", "#8c5f00", "#e4a400", "#d5c86f", "#939400", "#a7d380", "#138b00", "#42e087", "#00a86d", "#81c7a8", "#019a82", "#1eaaff", "#0268bb", "#5951d7", "#6b4570", "#ad20aa", "#ffa1e2", "#ff4a94"][0:n_clust]
color_list = ["#cc566a", "#cd6c39", "#a39440", "#64ac48", "#4aac8d", "#688bcd", "#8562cc", "#c361aa"]
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=n_clust)

num_planes= np.shape(roi_map)[0]
for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_clustered[i]
    roi_layer[0, 1:(n_clust+1)] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20][0:n_clust]
    roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    ax1[r, c].imshow(roi_layer, cmap=cm_roi)#rainbow")
    ax1[r, c].axis('off')
    ax1[r, c].set_title('z' + str(i))
    #print(roi_layer)

    
plt.show()
file_name = 'clusters_hrc_rois_210715_k' + str(n_clust) + '_2.jpg'
fig1.savefig(str(fish_dir/file_name), dpi=300)